# Objective
The objective of this analysis is to determine indicators of heavy traffic on the I-94.

The data used in this analysis has been sourced from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Metro+Interstate+Traffic+Volume#) and made avaiable by John Hogue.

In [2]:
import pandas as pd
traffic = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')

In [3]:
traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48204 entries, 0 to 48203
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              48204 non-null  object 
 1   temp                 48204 non-null  float64
 2   rain_1h              48204 non-null  float64
 3   snow_1h              48204 non-null  float64
 4   clouds_all           48204 non-null  int64  
 5   weather_main         48204 non-null  object 
 6   weather_description  48204 non-null  object 
 7   date_time            48204 non-null  object 
 8   traffic_volume       48204 non-null  int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 3.3+ MB


In [5]:
traffic.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,scattered clouds,2012-10-02 09:00:00,5545
1,None,289.36,0.0,0.0,75,Clouds,broken clouds,2012-10-02 10:00:00,4516
2,None,289.58,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 11:00:00,4767
3,None,290.13,0.0,0.0,90,Clouds,overcast clouds,2012-10-02 12:00:00,5026
4,None,291.14,0.0,0.0,75,Clouds,broken clouds,2012-10-02 13:00:00,4918


In [6]:
traffic.tail()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,date_time,traffic_volume
48199,None,283.45,0.0,0.0,75,Clouds,broken clouds,2018-09-30 19:00:00,3543
48200,None,282.76,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 20:00:00,2781
48201,None,282.73,0.0,0.0,90,Thunderstorm,proximity thunderstorm,2018-09-30 21:00:00,2159
48202,None,282.09,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 22:00:00,1450
48203,None,282.12,0.0,0.0,90,Clouds,overcast clouds,2018-09-30 23:00:00,954


# Data summary
The data set imformation mentions that the data is hourly Interstate 94 Westbound traffic volume for MN DoT ATR station 301, roughly midway between Minneapolis and St Paul, MN. It also mentions that hourly weather features and holidays are included for impacts on traffic volume.

### Data columns
- **holiday** Categorical US National holidays plus regional holiday, Minnesota State Fair 
- **temp** Numeric Average temp in kelvin 
- **rain_1h** Numeric Amount in mm of rain that occurred in the hour 
- **snow_1h** Numeric Amount in mm of snow that occurred in the hour 
- **clouds_all** Numeric Percentage of cloud cover 
- **weather_main** Categorical Short textual description of the current weather 
- **weather_description** Categorical Longer textual description of the current weather 
- **date_time** DateTime Hour of the data collected in local CST time 
- **traffic_volume** Numeric Hourly I-94 ATR 301 reported westbound traffic volume

# Data cleaning
From the info above we can see that we have 9 columns of data and 48,204 rows. My initial impression of the data is that it looks clean and does not have any missing values.

It is also notable that the column heading are all in snake-case